# Longtype
* 차량 쌍(TV, LV) 기준으로 정리하기
* 왜냐하면, 각 쌍별로 SSM이 계산되기 때문임

# Import

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
import os

from scipy import stats # Z-score를 이용한 이상값 제거

import math # arctangent; math.atan 사용 목적
import statistics

import pyarrow.parquet as pq
#df = pq.read_pandas('data.parquet').to_pandas()

# Load Dataset

In [4]:
working_dir = 'D:/OneDrive/Projects/2023_SSM_Feasibility/Dataset'

In [5]:
file_list = ['LV_trajectories-0750am-0805am.parquet']#, 'LV_trajectories-0805am-0820am.parquet', 'LV_trajectories-0820am-0835am.parquet']
folder_name = '02_processed'

for file, i in zip(file_list, [1]):
    file_path = os.path.join(working_dir, folder_name, file)
    globals()[f'df_{i}'] = pq.read_pandas(file_path).to_pandas()

In [6]:
df_1.head(3)

,veh_id,frm,local_y,local_x,V_len,V_wid,velocity,acc,lane,Preceeding,...,LVR_delta_velocity_y,LVR_acc_x,LVR_acc_y,LVR_Lane_record,LVR_Lane_record_split,LVR_Lane_change_direction,LV0_D,LVL_D,LVR_D,LC_CF
index,,,,,,,,,,,,,,,,,,,,,
0,2,13,17.375000,10.78125,4.417969,1.493164,43.90625,0.0,U02,0,...,-0.000847,0.004704,-0.002352,U04,U04_U04,Straight,NaN,NaN,10.132812,LC
1,2,14,17.375000,12.00000,4.417969,1.493164,43.90625,0.0,U02,0,...,0.000000,-0.002352,0.000000,U04,U04_U04,Straight,NaN,NaN,10.125000,LC
2,2,15,17.390625,13.21875,4.417969,1.493164,43.90625,0.0,U02,0,...,0.001694,-0.000000,0.004704,U04,U04_U04,Straight,NaN,NaN,10.132812,LC


# Pairing
* 차로를 막론하고, 모든 (타겟차량, 선행차량)의 페어링을 확보

In [ ]:
# for df, i in zip([df_1, df_2, df_3], [1, 2, 3]):

#     globals()[f'pairs_{i}'] = []
#     vehicle_list = list(df['veh_id'].unique()) # 각 파일의모든 타겟차량 리스트

#     for veh in tqdm(vehicle_list):
#         veh_df = df[df['veh_id'] == veh] # 각 차량

#         LV0_ID_list = list(veh_df['LV0_ID'].unique())
#         LVR_ID_list = list(veh_df['LVR_ID'].unique())
#         LVL_ID_list = list(veh_df['LVL_ID'].unique())

#         for LV0_ID in LV0_ID_list:    
#             if (pd.isna(LV0_ID) == False) and (LV0_ID != 0): # None 또는 np.NaN이 아니면 :: LV0이 있으면
#                 globals()[f'pairs_{i}'].append((veh, int(LV0_ID), 'LV0'))
#             else:
#                 pass

#         for LVR_ID in LVR_ID_list:    
#             if (pd.isna(LVR_ID) == False) and (LVR_ID != 0): # None 또는 np.NaN이 아니면
#                 globals()[f'pairs_{i}'].append((veh, int(LVR_ID), 'LVR'))
#             else:
#                 pass

#         for LVL_ID in LVL_ID_list:    
#             if (pd.isna(LVL_ID) == False) and (LVL_ID != 0): # None 또는 np.NaN이 아니면
#                 globals()[f'pairs_{i}'].append((veh, int(LVL_ID), 'LVL'))
#             else:
#                 pass

# Longtype
* pairs 리스트의 튜플(TV, LV)에 따라서 각 차량의 위치, 속도, 가속도 등을 df로부터 프레임별로 뽑아오기

In [ ]:
# def make_long_base(veh_id, LV_ID, LV_type, df):
#     """(TV, LV) 쌍의 프레임별 정보를 뽑아 데이터프레임으로 만들기 위해,
#     각 쌍별 프레임을 추출하여 베이스 DF를 만들어주는 함수"""
    
#     target_df = df[df['veh_id'] == veh_id].copy()
#     target_frames = list(target_df['frm'].unique())
    
#     long_base = pd.DataFrame({'frm' : target_frames})
#     long_base['veh_id'] = veh_id
#     long_base['LV_ID'] = LV_ID
#     long_base['LV_type'] = LV_type
    
#     long_base = long_base[['veh_id', 'LV_ID', 'LV_type', 'frm']]
    
#     return long_base

In [7]:
for df, i in tqdm(zip([df_1], [1])):

    df_LV0 = df[['veh_id', 'LV0_ID', 'frm']].copy()
    df_LV0 = df_LV0.rename({'LV0_ID' : 'LV_ID'}, axis =1)
    df_LV0['LV_type'] = 'LV0'
    
    df_LVL = df[['veh_id', 'LVL_ID', 'frm']].copy()
    df_LVL = df_LVL.rename({'LVL_ID' : 'LV_ID'}, axis = 1)
    df_LVL['LV_type'] = 'LVL'

    df_LVR = df[['veh_id', 'LVR_ID', 'frm']].copy()
    df_LVR = df_LVR.rename({'LVR_ID' : 'LV_ID'}, axis = 1)
    df_LVR['LV_type'] = 'LVR'
    
    total_long = pd.concat([df_LV0, df_LVL, df_LVR])
    
    # LV_ID가 0인 것은 제거한다
    total_long = total_long[total_long['LV_ID'] != 0]
    total_long.reset_index(inplace = True, drop = True)
    
    globals()[f'long_{i}'] = total_long

1it [00:00,  4.68it/s]


In [8]:
long_1.head(3)

,veh_id,LV_ID,frm,LV_type
0,6,4,141,LV0
1,6,4,142,LV0
2,6,4,143,LV0


In [10]:
warnings.filterwarnings('ignore')

target_df_cols = ['local_x', 'local_y', 'V_len', 'V_wid', 'velocity', 'acc', 'lane', 'Time',
                  'local_x_before', 'local_y_before', 'delta_local_x', 'delta_local_y',
                  'velocity_x', 'velocity_y', 'velocity_x_before', 'velocity_y_before', 'delta_velocity_x', 'delta_velocity_y',
                  'acc_x', 'acc_y', 'lane_past', 'Lane_record', 'Lane_record_split', 'Lane_00', 'Lane_99',
                  'Lane_change', 'Lane_leave', 'Lane_change_direction', 'LC_CF']

save_folder = '02_processed_long'
save_files = ['LV_long_trajectories-0750am-0805am.parquet', 'LV_long_trajectories-0805am-0820am.parquet', 'LV_long_trajectories-0820am-0835am.parquet']

for long, i in tqdm(zip([long_1], [1])):

    reference_df = globals()[f'df_{i}'][['veh_id', 'frm'] + target_df_cols].copy() # Merge의 reference가 될 데이터프레임

    long_TV = pd.merge(reference_df, long, how = 'left', left_on = ['veh_id', 'frm'], right_on = ['veh_id', 'frm'])
    long_LV = pd.merge(reference_df, long, how = 'left', left_on = ['veh_id', 'frm'], right_on = ['LV_ID', 'frm']).drop(['veh_id_x', 'veh_id_y'], axis = 1)

    # LV 데이터프레임 컬럼이름을 적절하게 변경
    for col in target_df_cols:
        long_LV = long_LV.rename({col : 'LV_'+col}, axis = 1)

    long_LV.drop(['LV_ID', 'frm', 'LV_type'], axis = 1, inplace = True)

    long_tot = pd.concat([long_TV, long_LV], axis = 1)

    # 거리관련 변수
    long_tot['D_x'] = long_tot['LV_local_x'] - long_tot['local_x']
    long_tot['D_y'] = long_tot['LV_local_y'] - long_tot['local_y']
    long_tot['D'] = (long_tot['D_x']**2 + long_tot['D_y']**2) ** (1/2)
    long_tot.reset_index(inplace = True, drop = True)
    
    long_tot['D_lag'] = long_tot['D'].shift(1) # 직전 row의 값임
    long_tot['veh_id'] = long_tot['veh_id'].astype('str')
    long_tot['LV_ID'] = long_tot['LV_ID'].astype('str')
    long_tot['veh_id_lag'] = long_tot['veh_id'].shift(1) # 직전 row 값
    long_tot['LV_ID_lag'] = long_tot['LV_ID'].shift(1) # 직전 row 값
    
    long_tot['D_gap'] = long_tot['D'] - long_tot['D_lag'] # 거리 변화. (-)이면 줄어들고 있다는 뜻이다
    
    # 만약 veh_id, LV_id 의 lag값이 현재 row와 다르다면, 새로운 값이 시작된 것이므로 D_gap을 NaN으로 만들어줘야 한다.
    long_tot.loc[(long_tot['veh_id'] != long_tot['veh_id_lag']) | (long_tot['LV_ID'] != long_tot['LV_ID_lag']), 'D_gap'] = None
        
    #print(f'{i} : {len(long_final)}')
    long_tot.reset_index(inplace = True, drop = True)
    long_tot.drop(['D_lag', 'veh_id_lag', 'LV_ID_lag'], axis = 1, inplace = True)
    
    # 저장하기
    save_path = os.path.join(working_dir, save_folder, save_files[i-1])
    long_tot.to_parquet(save_path, engine = 'fastparquet', compression = 'gzip') 

1it [00:40, 40.78s/it]


In [11]:
long_tot

,veh_id,frm,local_x,local_y,V_len,V_wid,velocity,acc,lane,Time,...,LV_Lane_00,LV_Lane_99,LV_Lane_change,LV_Lane_leave,LV_Lane_change_direction,LV_LC_CF,D_x,D_y,D,D_gap
0,2,13,10.781250,17.375000,4.417969,1.493164,43.90625,0.000000,U02,0.166667,...,U02,U01,None,None,Left,LC,0.0,0.0,0.0,NaN
1,2,14,12.000000,17.375000,4.417969,1.493164,43.90625,0.000000,U02,0.200000,...,U02,U01,None,None,Left,LC,0.0,0.0,0.0,0.0
2,2,15,13.218750,17.390625,4.417969,1.493164,43.90625,0.000000,U02,0.233333,...,U02,U01,None,None,Left,LC,0.0,0.0,0.0,0.0
3,2,16,14.445312,17.390625,4.417969,1.493164,43.90625,0.000000,U02,0.266667,...,U02,U01,None,None,Left,LC,0.0,0.0,0.0,0.0
4,2,17,15.664062,17.406250,4.417969,1.493164,43.90625,0.000000,U02,0.300000,...,U02,U01,None,None,Left,LC,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734770,2783,8700,582.000000,5.617188,5.335938,1.645508,38.50000,-0.044586,U05,289.733333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2734771,2783,8701,583.000000,5.589844,5.335938,1.645508,38.46875,0.000000,U05,289.766667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2734772,2783,8701,583.000000,5.589844,5.335938,1.645508,38.46875,0.000000,U05,289.766667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2734773,2783,8702,584.000000,5.578125,5.335938,1.645508,38.31250,-0.201660,U05,289.800000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
